In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data
df = pd.read_csv("../data/combine.csv")
df.head()

,Date,Close,ma_5,ma_10,ma_20,ma_30,ma_60,ma_90,ma_180,ma_240,...,Nasdaq_diff,SP500_diff,INDU_diff,Oil_diff,Btc_diffpct,Gold_diffpct,Nasdaq_diffpct,SP500_diffpct,INDU_diffpct,Oil_diffpct
0,2012-01-03,5.32,4.975,4.975000,4.975000,4.975000,4.975000,4.975000,4.975000,4.975000,...,0.00,0.00,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2012-01-04,5.57,5.094,5.094000,5.094000,5.094000,5.094000,5.094000,5.094000,5.094000,...,-0.36,0.24,21.04,0.8793,0.046992,0.014944,-0.000136,0.000188,0.001697,0.019997
2,2012-01-05,6.65,5.508,5.353333,5.353333,5.353333,5.353333,5.353333,5.353333,5.353333,...,21.50,3.76,-2.72,0.0000,0.193896,-0.004148,0.008118,0.002944,-0.000219,0.000000
3,2012-01-06,6.90,5.888,5.574286,5.574286,5.574286,5.574286,5.574286,5.574286,5.574286,...,4.36,-3.25,-55.78,0.0000,0.037594,-0.009996,0.001633,-0.002537,-0.004493,0.000000
4,2012-01-09,6.90,6.850,5.972000,5.972000,5.972000,5.972000,5.972000,5.972000,5.972000,...,2.34,2.89,32.77,0.0000,0.000000,0.003576,0.000875,0.002262,0.002651,0.000000


In [3]:
# define to check if close price increased compare to the previous close
df["Trend"] = df.apply(lambda x: 1 if x["Btc_diff"] > 0 else (2 if x["Btc_diff"] <0 else 0), axis=1)

In [4]:
df[["Date", "Trend"]]

,Date,Trend
0,2012-01-03,0
1,2012-01-04,1
2,2012-01-05,1
3,2012-01-06,1
4,2012-01-09,0
...,...,...
2318,2021-03-25,2
2319,2021-03-26,1
2320,2021-03-29,1
2321,2021-03-30,1


In [5]:
ind_train_start = 360
ind_test_start = int((len(df) - 360) * 0.8)

In [6]:
df.columns

Index(['Date', 'Close', 'ma_5', 'ma_10', 'ma_20', 'ma_30', 'ma_60', 'ma_90',
       'ma_180', 'ma_240', 'ma_360', 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU',
       'Oil', 'Btc_diff', 'Gold_diff', 'Nasdaq_diff', 'SP500_diff',
       'INDU_diff', 'Oil_diff', 'Btc_diffpct', 'Gold_diffpct',
       'Nasdaq_diffpct', 'SP500_diffpct', 'INDU_diffpct', 'Oil_diffpct',
       'Trend'],
      dtype='object')

In [7]:
# change here
X = df[['Close' , 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU', 'Oil']]
y = df[["Trend"]]

In [8]:
X_train = X.iloc[ind_train_start:ind_test_start]
X_test = X.iloc[ind_test_start:]
X_test = X_test.iloc[:len(X_test)-1]
X_test.iloc[0]

# for using close price
#y_train = y.iloc[ind_train_start + 1:ind_test_start + 1]
#y_test = y.iloc[ind_test_start+1:]
# for using other rows
y_train = y.iloc[ind_train_start+1:ind_test_start+1]
y_test = y.iloc[ind_test_start+1:]

In [9]:
print(X_train.shape, y_train.shape)

(1210, 6) (1210, 1)


In [10]:
ind_test = ind_test_start + 3
ind_test2 = ind_test - ind_test_start
print(df[["Date", "Close", 'Gold', 'Nasdaq_Comp', 'SP500', 'INDU', 'Oil', "Btc_diff", "Trend"]].iloc[ind_test])
print(X_test.iloc[ind_test2])
print(y_test.iloc[ind_test2])

Date           2018-04-10
Close              6880.0
Gold                12.77
Nasdaq_Comp        7094.3
SP500             2656.87
INDU              24408.0
Oil               18.5393
Btc_diff          -289.99
Trend                   2
Name: 1573, dtype: object
Close           6880.0000
Gold              12.7700
Nasdaq_Comp     7094.3000
SP500           2656.8700
INDU           24408.0000
Oil               18.5393
Name: 1573, dtype: float64
Trend    1
Name: 1574, dtype: int64


In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
X_scaler = StandardScaler().fit(X)

In [13]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# for predict price
from tensorflow.keras.utils import to_categorical

In [15]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [16]:
print(X_train_scaled, y_train_categorical)

[[-0.54605436  0.0365894  -1.02765551 -1.08271874 -0.99454033  1.23129059]
 [-0.54570171  0.04996446 -1.04202584 -1.10414619 -1.01866292  1.23129059]
 [-0.54600382  0.05442282 -1.02434231 -1.06661273 -0.98425508  1.29250421]
 ...
 [ 0.27663567 -0.74696659  0.32326902  0.42200447  0.6268557  -0.50653828]
 [ 0.32356145 -0.76034166  0.3512699   0.47328235  0.70089772 -0.50653828]
 [ 0.31418547 -0.75811248  0.39094568  0.5208919   0.7448355  -0.49627758]] [[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [17]:
print(X_train_scaled.shape, y_train_categorical.shape)

(1210, 6) (1210, 3)


In [18]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [19]:
from tensorflow.keras.layers import Dense
number_inputs = len(X.columns)
model.add(Dense(units=30, activation='relu', input_dim=number_inputs))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                210       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 93        
Total params: 1,233
Trainable params: 1,233
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [22]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
38/38 - 0s - loss: 0.8968 - accuracy: 0.5008
Epoch 2/1000
38/38 - 0s - loss: 0.7550 - accuracy: 0.5669
Epoch 3/1000
38/38 - 0s - loss: 0.7206 - accuracy: 0.5876
Epoch 4/1000
38/38 - 0s - loss: 0.7105 - accuracy: 0.5884
Epoch 5/1000
38/38 - 0s - loss: 0.7068 - accuracy: 0.5950
Epoch 6/1000
38/38 - 0s - loss: 0.7040 - accuracy: 0.5926
Epoch 7/1000
38/38 - 0s - loss: 0.7022 - accuracy: 0.5876
Epoch 8/1000
38/38 - 0s - loss: 0.7020 - accuracy: 0.5851
Epoch 9/1000
38/38 - 0s - loss: 0.6996 - accuracy: 0.5926
Epoch 10/1000
38/38 - 0s - loss: 0.6997 - accuracy: 0.5934
Epoch 11/1000
38/38 - 0s - loss: 0.6976 - accuracy: 0.6008
Epoch 12/1000
38/38 - 0s - loss: 0.6974 - accuracy: 0.5893
Epoch 13/1000
38/38 - 0s - loss: 0.6958 - accuracy: 0.6033
Epoch 14/1000
38/38 - 0s - loss: 0.6969 - accuracy: 0.5950
Epoch 15/1000
38/38 - 0s - loss: 0.6943 - accuracy: 0.5926
Epoch 16/1000
38/38 - 0s - loss: 0.6947 - accuracy: 0.6066
Epoch 17/1000
38/38 - 0s - loss: 0.6931 - accuracy: 0.5992
Epoch 

Epoch 140/1000
38/38 - 0s - loss: 0.6700 - accuracy: 0.6240
Epoch 141/1000
38/38 - 0s - loss: 0.6702 - accuracy: 0.6273
Epoch 142/1000
38/38 - 0s - loss: 0.6739 - accuracy: 0.6174
Epoch 143/1000
38/38 - 0s - loss: 0.6722 - accuracy: 0.6132
Epoch 144/1000
38/38 - 0s - loss: 0.6734 - accuracy: 0.6240
Epoch 145/1000
38/38 - 0s - loss: 0.6703 - accuracy: 0.6182
Epoch 146/1000
38/38 - 0s - loss: 0.6711 - accuracy: 0.6240
Epoch 147/1000
38/38 - 0s - loss: 0.6694 - accuracy: 0.6240
Epoch 148/1000
38/38 - 0s - loss: 0.6704 - accuracy: 0.6240
Epoch 149/1000
38/38 - 0s - loss: 0.6702 - accuracy: 0.6240
Epoch 150/1000
38/38 - 0s - loss: 0.6692 - accuracy: 0.6248
Epoch 151/1000
38/38 - 0s - loss: 0.6710 - accuracy: 0.6306
Epoch 152/1000
38/38 - 0s - loss: 0.6698 - accuracy: 0.6198
Epoch 153/1000
38/38 - 0s - loss: 0.6689 - accuracy: 0.6182
Epoch 154/1000
38/38 - 0s - loss: 0.6713 - accuracy: 0.6322
Epoch 155/1000
38/38 - 0s - loss: 0.6692 - accuracy: 0.6289
Epoch 156/1000
38/38 - 0s - loss: 0.6692

Epoch 277/1000
38/38 - 0s - loss: 0.6653 - accuracy: 0.6306
Epoch 278/1000
38/38 - 0s - loss: 0.6622 - accuracy: 0.6264
Epoch 279/1000
38/38 - 0s - loss: 0.6615 - accuracy: 0.6281
Epoch 280/1000
38/38 - 0s - loss: 0.6619 - accuracy: 0.6256
Epoch 281/1000
38/38 - 0s - loss: 0.6648 - accuracy: 0.6240
Epoch 282/1000
38/38 - 0s - loss: 0.6628 - accuracy: 0.6215
Epoch 283/1000
38/38 - 0s - loss: 0.6622 - accuracy: 0.6223
Epoch 284/1000
38/38 - 0s - loss: 0.6610 - accuracy: 0.6207
Epoch 285/1000
38/38 - 0s - loss: 0.6646 - accuracy: 0.6273
Epoch 286/1000
38/38 - 0s - loss: 0.6622 - accuracy: 0.6281
Epoch 287/1000
38/38 - 0s - loss: 0.6628 - accuracy: 0.6240
Epoch 288/1000
38/38 - 0s - loss: 0.6635 - accuracy: 0.6231
Epoch 289/1000
38/38 - 0s - loss: 0.6625 - accuracy: 0.6256
Epoch 290/1000
38/38 - 0s - loss: 0.6632 - accuracy: 0.6289
Epoch 291/1000
38/38 - 0s - loss: 0.6600 - accuracy: 0.6240
Epoch 292/1000
38/38 - 0s - loss: 0.6620 - accuracy: 0.6256
Epoch 293/1000
38/38 - 0s - loss: 0.6633

Epoch 414/1000
38/38 - 0s - loss: 0.6539 - accuracy: 0.6421
Epoch 415/1000
38/38 - 0s - loss: 0.6550 - accuracy: 0.6372
Epoch 416/1000
38/38 - 0s - loss: 0.6542 - accuracy: 0.6281
Epoch 417/1000
38/38 - 0s - loss: 0.6555 - accuracy: 0.6355
Epoch 418/1000
38/38 - 0s - loss: 0.6560 - accuracy: 0.6264
Epoch 419/1000
38/38 - 0s - loss: 0.6543 - accuracy: 0.6289
Epoch 420/1000
38/38 - 0s - loss: 0.6544 - accuracy: 0.6322
Epoch 421/1000
38/38 - 0s - loss: 0.6541 - accuracy: 0.6388
Epoch 422/1000
38/38 - 0s - loss: 0.6542 - accuracy: 0.6264
Epoch 423/1000
38/38 - 0s - loss: 0.6540 - accuracy: 0.6314
Epoch 424/1000
38/38 - 0s - loss: 0.6527 - accuracy: 0.6347
Epoch 425/1000
38/38 - 0s - loss: 0.6561 - accuracy: 0.6264
Epoch 426/1000
38/38 - 0s - loss: 0.6556 - accuracy: 0.6314
Epoch 427/1000
38/38 - 0s - loss: 0.6573 - accuracy: 0.6273
Epoch 428/1000
38/38 - 0s - loss: 0.6601 - accuracy: 0.6231
Epoch 429/1000
38/38 - 0s - loss: 0.6537 - accuracy: 0.6289
Epoch 430/1000
38/38 - 0s - loss: 0.6551

Epoch 551/1000
38/38 - 0s - loss: 0.6491 - accuracy: 0.6364
Epoch 552/1000
38/38 - 0s - loss: 0.6471 - accuracy: 0.6314
Epoch 553/1000
38/38 - 0s - loss: 0.6471 - accuracy: 0.6314
Epoch 554/1000
38/38 - 0s - loss: 0.6466 - accuracy: 0.6306
Epoch 555/1000
38/38 - 0s - loss: 0.6459 - accuracy: 0.6364
Epoch 556/1000
38/38 - 0s - loss: 0.6466 - accuracy: 0.6479
Epoch 557/1000
38/38 - 0s - loss: 0.6490 - accuracy: 0.6355
Epoch 558/1000
38/38 - 0s - loss: 0.6469 - accuracy: 0.6322
Epoch 559/1000
38/38 - 0s - loss: 0.6446 - accuracy: 0.6347
Epoch 560/1000
38/38 - 0s - loss: 0.6461 - accuracy: 0.6355
Epoch 561/1000
38/38 - 0s - loss: 0.6470 - accuracy: 0.6397
Epoch 562/1000
38/38 - 0s - loss: 0.6455 - accuracy: 0.6471
Epoch 563/1000
38/38 - 0s - loss: 0.6477 - accuracy: 0.6364
Epoch 564/1000
38/38 - 0s - loss: 0.6479 - accuracy: 0.6306
Epoch 565/1000
38/38 - 0s - loss: 0.6477 - accuracy: 0.6413
Epoch 566/1000
38/38 - 0s - loss: 0.6456 - accuracy: 0.6314
Epoch 567/1000
38/38 - 0s - loss: 0.6451

Epoch 688/1000
38/38 - 0s - loss: 0.6402 - accuracy: 0.6413
Epoch 689/1000
38/38 - 0s - loss: 0.6460 - accuracy: 0.6463
Epoch 690/1000
38/38 - 0s - loss: 0.6392 - accuracy: 0.6463
Epoch 691/1000
38/38 - 0s - loss: 0.6455 - accuracy: 0.6331
Epoch 692/1000
38/38 - 0s - loss: 0.6464 - accuracy: 0.6455
Epoch 693/1000
38/38 - 0s - loss: 0.6411 - accuracy: 0.6430
Epoch 694/1000
38/38 - 0s - loss: 0.6433 - accuracy: 0.6438
Epoch 695/1000
38/38 - 0s - loss: 0.6424 - accuracy: 0.6364
Epoch 696/1000
38/38 - 0s - loss: 0.6414 - accuracy: 0.6314
Epoch 697/1000
38/38 - 0s - loss: 0.6407 - accuracy: 0.6388
Epoch 698/1000
38/38 - 0s - loss: 0.6416 - accuracy: 0.6380
Epoch 699/1000
38/38 - 0s - loss: 0.6434 - accuracy: 0.6347
Epoch 700/1000
38/38 - 0s - loss: 0.6408 - accuracy: 0.6388
Epoch 701/1000
38/38 - 0s - loss: 0.6430 - accuracy: 0.6380
Epoch 702/1000
38/38 - 0s - loss: 0.6442 - accuracy: 0.6413
Epoch 703/1000
38/38 - 0s - loss: 0.6402 - accuracy: 0.6455
Epoch 704/1000
38/38 - 0s - loss: 0.6389

Epoch 825/1000
38/38 - 0s - loss: 0.6364 - accuracy: 0.6504
Epoch 826/1000
38/38 - 0s - loss: 0.6374 - accuracy: 0.6355
Epoch 827/1000
38/38 - 0s - loss: 0.6368 - accuracy: 0.6438
Epoch 828/1000
38/38 - 0s - loss: 0.6397 - accuracy: 0.6347
Epoch 829/1000
38/38 - 0s - loss: 0.6372 - accuracy: 0.6397
Epoch 830/1000
38/38 - 0s - loss: 0.6364 - accuracy: 0.6471
Epoch 831/1000
38/38 - 0s - loss: 0.6344 - accuracy: 0.6488
Epoch 832/1000
38/38 - 0s - loss: 0.6373 - accuracy: 0.6331
Epoch 833/1000
38/38 - 0s - loss: 0.6344 - accuracy: 0.6512
Epoch 834/1000
38/38 - 0s - loss: 0.6365 - accuracy: 0.6430
Epoch 835/1000
38/38 - 0s - loss: 0.6360 - accuracy: 0.6430
Epoch 836/1000
38/38 - 0s - loss: 0.6462 - accuracy: 0.6298
Epoch 837/1000
38/38 - 0s - loss: 0.6384 - accuracy: 0.6438
Epoch 838/1000
38/38 - 0s - loss: 0.6357 - accuracy: 0.6446
Epoch 839/1000
38/38 - 0s - loss: 0.6353 - accuracy: 0.6471
Epoch 840/1000
38/38 - 0s - loss: 0.6339 - accuracy: 0.6455
Epoch 841/1000
38/38 - 0s - loss: 0.6366

Epoch 962/1000
38/38 - 0s - loss: 0.6325 - accuracy: 0.6471
Epoch 963/1000
38/38 - 0s - loss: 0.6331 - accuracy: 0.6413
Epoch 964/1000
38/38 - 0s - loss: 0.6320 - accuracy: 0.6372
Epoch 965/1000
38/38 - 0s - loss: 0.6326 - accuracy: 0.6521
Epoch 966/1000
38/38 - 0s - loss: 0.6363 - accuracy: 0.6421
Epoch 967/1000
38/38 - 0s - loss: 0.6330 - accuracy: 0.6479
Epoch 968/1000
38/38 - 0s - loss: 0.6320 - accuracy: 0.6438
Epoch 969/1000
38/38 - 0s - loss: 0.6330 - accuracy: 0.6397
Epoch 970/1000
38/38 - 0s - loss: 0.6301 - accuracy: 0.6471
Epoch 971/1000
38/38 - 0s - loss: 0.6334 - accuracy: 0.6504
Epoch 972/1000
38/38 - 0s - loss: 0.6329 - accuracy: 0.6438
Epoch 973/1000
38/38 - 0s - loss: 0.6302 - accuracy: 0.6463
Epoch 974/1000
38/38 - 0s - loss: 0.6340 - accuracy: 0.6496
Epoch 975/1000
38/38 - 0s - loss: 0.6303 - accuracy: 0.6455
Epoch 976/1000
38/38 - 0s - loss: 0.6348 - accuracy: 0.6488
Epoch 977/1000
38/38 - 0s - loss: 0.6297 - accuracy: 0.6479
Epoch 978/1000
38/38 - 0s - loss: 0.6318

In [23]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

24/24 - 0s - loss: 3.1089 - accuracy: 0.5066
Normal Neural Network - Loss: 3.1089348793029785, Accuracy: 0.5066489577293396
